In [1]:
# On the instance where you are running jupyter,
# authenticate with gcloud first:
#
#     gcloud auth application-default login

import bigframes.pandas as bpd

# Change this location to the location of your datasets.
# We use "us" as that is the location of the sample data.
bpd.options.bigquery.location = "us"

In [2]:
df = bpd.read_gbq(
    "bigquery-public-data.chicago_taxi_trips.taxi_trips",
    # For large tables, a unique index or multiindex is highly recommended.
    # This prevents BigQuery DataFrame from creating a default sequential index.
    index_col=["trip_start_timestamp", "unique_key"],
)

In [3]:
type(df)

bigframes.dataframe.DataFrame

In [4]:
df.head(n=1)

,,taxi_id,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
trip_start_timestamp,unique_key,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,006d53094ff5b3c81140c00aa91fdb5467db0802,4d924c1ad8126ad7efcbe49964b63ce4e62e11223b0f41...,2013-01-01 00:15:00+00:00,360,2.3,17031330100,17031320100,33,32,7.85,0.0,...,2.0,9.85,Cash,<NA>,41.85935,-87.617358,POINT (-87.6173580061 41.859349715),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918)


In [5]:
df.tail(n=1)

,,taxi_id,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
trip_start_timestamp,unique_key,,,,,,,,,,,,,,,,,,,,,
2023-07-01 00:00:00+00:00,fb004dfe98302ccf34e5e11bf1081568a94843a2,75cf3a53aae5e5858361a7ca64f75d3407dc0a44d7bc42...,2023-07-01 00:00:00+00:00,562,1.82,<NA>,<NA>,22,22,9.5,2.53,...,0.0,12.03,Mobile,City Service,41.922761,-87.699155,POINT (-87.6991553432 41.9227606205),41.922761,-87.699155,POINT (-87.6991553432 41.9227606205)


In [6]:
df = df[[
    "company",
    "trip_miles",
    "fare",
    "tips",
]]

In [7]:
df

company  \
trip_start_timestamp      unique_key                                                                    
2013-01-01 00:00:00+00:00 006d53094ff5b3c81140c00aa91fdb5467db0802                               <NA>   
                          00a723cf08cf29c56faa8ef77be519f6590b3405                               <NA>   
                          01396f0debefe45ccce23e48dd471edd67bd32e8          Dispatch Taxi Affiliation   
                          01e9a03fd793670ed35ef7195eeb99775895611f          Dispatch Taxi Affiliation   
                          024f0f183c43ad2be33f232fd8dca0a86a3e4925                               <NA>   
                          04373ef74229c2122ff90d92642fcd94c88302e4          Dispatch Taxi Affiliation   
                          0501a22fba176594d6f974360e20737af1b6cb66                               <NA>   
                          05faff951f2a2703fe3eb24efac026c2a1669a92                               <NA>   
                          0651a5e1eb4792a53395c3b3b4fd9f7f8383cfbe  Blue Ribbon Taxi Association Inc.   
                          06b4dc6dcd6295e69284e1c9d5a98015b0588ea6            Choice Taxi Association   
                          07483a0baa6df0af68696b6f2483b7f429ff1827           Northwest Management LLC   
                          07585d085f65cd512b94de09684253fe0ffb31a2          Taxi Affiliation Services   
                          090ea9dc5dc26374ee06cdab1b809895beb1befc          Dispatch Taxi Affiliation   
                          091eda4ec41520d6ba96003f05eb10744f73aa82           Northwest Management LLC   
                          097f2efed248cb5900792a89bba96a64918345ce  Blue Ribbon Taxi Association Inc.   
                          09ffa3f40d36dbf8d51ca903738d9bba92557ed7          Taxi Affiliation Services   
                          0a3446fb4f72d6485548883cd7f6cfe068d44829          Taxi Affiliation Services   
                          0a7ea707a8700555e898489200a66b546a834170                               <NA>   
                          0be545d9934dd343a2d929fca2e53d3d2851137b          Dispatch Taxi Affiliation   
                          0c2b3705c9748a5949bf5b7d727b2d1f2e18805f                               <NA>   
                          0d37bbfeb207fbb2353e1911fdf878f3c24ed4c4          Taxi Affiliation Services   
                          0dafcce34426b5377fd3925418a4c24b3e45c7d9                               <NA>   
                          0f5f7162b289dd9743b9d2e7fc75bf69696bb7bc          Taxi Affiliation Services   
                          0f63007e437d589c9139acb026f03a832da3240f           Northwest Management LLC   
                          10508745a1069e9cdaad8760a6a243dc924cc0f2                               <NA>   

                                                                    trip_miles  \
trip_start_timestamp      unique_key                                             
2013-01-01 00:00:00+00:00 006d53094ff5b3c81140c00aa91fdb5467db0802         2.3   
                          00a723cf08cf29c56faa8ef77be519f6590b3405         1.0   
                          01396f0debefe45ccce23e48dd471edd67bd32e8         0.0   
                          01e9a03fd793670ed35ef7195eeb99775895611f         1.8   
                          024f0f183c43ad2be33f232fd8dca0a86a3e4925         4.9   
                          04373ef74229c2122ff90d92642fcd94c88302e4         0.0   
                          0501a22fba176594d6f974360e20737af1b6cb66         2.2   
                          05faff951f2a2703fe3eb24efac026c2a1669a92         3.3   
                          0651a5e1eb4792a53395c3b3b4fd9f7f8383cfbe         0.0   
                          06b4dc6dcd6295e69284e1c9d5a98015b0588ea6         0.0   
                          07483a0baa6df0af68696b6f2483b7f429ff1827         0.6   
                          07585d085f65cd512b94de09684253fe0ffb31a2         7.0   
                          090ea9dc5dc26374ee06cdab1b809895beb1befc         0.0   
                          09

In [8]:
df['fare'].head(n=10)

trip_start_timestamp       unique_key                              
2013-01-01 00:00:00+00:00  006d53094ff5b3c81140c00aa91fdb5467db0802     7.85
                           00a723cf08cf29c56faa8ef77be519f6590b3405     5.65
                           01396f0debefe45ccce23e48dd471edd67bd32e8     8.85
                           01e9a03fd793670ed35ef7195eeb99775895611f     7.45
                           024f0f183c43ad2be33f232fd8dca0a86a3e4925    14.05
                           04373ef74229c2122ff90d92642fcd94c88302e4     6.65
                           0501a22fba176594d6f974360e20737af1b6cb66     7.65
                           05faff951f2a2703fe3eb24efac026c2a1669a92     9.65
                           0651a5e1eb4792a53395c3b3b4fd9f7f8383cfbe     6.25
                           06b4dc6dcd6295e69284e1c9d5a98015b0588ea6    13.85
Name: fare, dtype: Float64

In [9]:
df['fare'].mean()

13.669183161039143

In [10]:
subtotal = df['fare'] + df['tips']
subtotal.head()

trip_start_timestamp       unique_key                              
2013-01-01 00:00:00+00:00  006d53094ff5b3c81140c00aa91fdb5467db0802     7.85
                           00a723cf08cf29c56faa8ef77be519f6590b3405     5.65
                           01396f0debefe45ccce23e48dd471edd67bd32e8     8.85
                           01e9a03fd793670ed35ef7195eeb99775895611f     7.45
                           024f0f183c43ad2be33f232fd8dca0a86a3e4925    14.05
dtype: Float64

In [11]:
subtotal[df['company'] == 'Norshore Cab'].sum()

2844629.7599999933